In [9]:
%%capture
%pip install torch torchvision h5py xarray matplotlib netcdf4

In [10]:
import random
import time
import os
import datetime
import itertools
import math
import sys

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.utils.data as data
from torch.utils.data import DataLoader
import numpy as np
import h5py
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr

sys.path.append('../common')
import common
from common import VideoDataset
from ConvLSTMmodels import ConvLSTM, EncoderRNN, ConvLSTMEncoderDecoder

In [11]:
# check tensorflow version
print("pytorch version:", torch.__version__)
# check available gpu
if torch.cuda.is_available():
    gpus =  torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Available GPUs: {gpus}")
else:
    print("GPUs not available. Only CPU.")

pytorch version: 2.3.1+cu121
GPUs not available. Only CPU.


# Reading the data

## Analyzing the data

In [12]:
cwd = os.getcwd()
pardir = os.path.dirname(os.path.dirname(cwd))
data_folder = os.path.join(pardir,'data')
data_path = os.path.join(data_folder,'video_prediction_dataset.hdf5')
model_name = 'ConvLSTM'
output_folder = os.path.join(cwd,"save", model_name)
if not os.path.isdir(output_folder):
    os.makedirs(output_folder)
print("data_folder:", data_folder)
print("data_path:", data_path)
print("output_folder:", output_folder)

data_folder: /home/javgat/Repos/Otros/presente/data
data_path: /home/javgat/Repos/Otros/presente/data/video_prediction_dataset.hdf5
output_folder: /home/javgat/Repos/Otros/presente/codes/ConvLSTM/save/ConvLSTM


In [13]:
def get_all(name):
    # show structure of the hdf5 data
    if name!=None:
        print(forecast_dataset[name])

# generate handler for the hdf5 data
with h5py.File(data_path, 'r') as forecast_dataset:
    forecast_dataset.visit(get_all)

<HDF5 group "/test" (4 members)>
<HDF5 dataset "images_log": shape (4467, 16, 64, 64, 3), type "|u1">
<HDF5 dataset "images_pred": shape (4467, 15, 64, 64, 3), type "|u1">
<HDF5 dataset "pv_log": shape (4467, 16), type "<f4">
<HDF5 dataset "pv_pred": shape (4467, 15), type "<f4">
<HDF5 group "/trainval" (4 members)>
<HDF5 dataset "images_log": shape (53336, 16, 64, 64, 3), type "|u1">
<HDF5 dataset "images_pred": shape (53336, 15, 64, 64, 3), type "|u1">
<HDF5 dataset "pv_log": shape (53336, 16), type "<f4">
<HDF5 dataset "pv_pred": shape (53336, 15), type "<f4">


In [14]:
with h5py.File(data_path, 'r') as fds:
    group_names = list(fds.keys())
    print(group_names)

dss = {}
for gname in group_names:
    dss[gname] = xr.open_dataset(data_path, group=gname)

['test', 'trainval']


In [15]:
print(dss['test']['images_log'])
print(dss['test']['images_log'].shape)

<xarray.DataArray 'images_log' (phony_dim_0: 4467, phony_dim_1: 16,
                                phony_dim_2: 64, phony_dim_3: 64, phony_dim_4: 3)> Size: 878MB
[878247936 values with dtype=uint8]
Dimensions without coordinates: phony_dim_0, phony_dim_1, phony_dim_2,
                                phony_dim_3, phony_dim_4
(4467, 16, 64, 64, 3)


In [16]:
times_curr_train = np.load(os.path.join(data_folder,"times_curr_trainval.npy"),allow_pickle=True)
times_curr_test = np.load(os.path.join(data_folder,"times_curr_test.npy"),allow_pickle=True)

print('-'*50)
print("times_curr_train.shape:", times_curr_train.shape)
print("images_log_train.shape:", dss['trainval']['images_log'].shape)
print("images_pred_train.shape:", dss['trainval']['images_pred'].shape)
print("times_curr_test.shape:", times_curr_test.shape)
print("images_log_test.shape:", dss['test']['images_log'].shape)
print("images_pred_test.shape:", dss['test']['images_pred'].shape)
print('-'*50)

# get the input dimension for constructing the model
num_log_frame = dss['trainval']['images_log'].shape[1]
img_side_len = dss['trainval']['images_log'].shape[2]
num_color_channel = dss['trainval']['images_log'].shape[4]
num_pred_frame = dss['trainval']['images_pred'].shape[1]
image_log_dim = [num_log_frame,img_side_len,img_side_len,num_color_channel]
image_pred_dim = [num_pred_frame,img_side_len,img_side_len,num_color_channel]

print("image side length:", img_side_len)
print("number of log frames:", num_log_frame)
print("number of pred frames:", num_pred_frame)
print("number of color channels:", num_color_channel)
print("context(log) image dimension:", image_log_dim)
print("future(pred) image dimension:", image_pred_dim)

--------------------------------------------------
times_curr_train.shape: (53336,)
images_log_train.shape: (53336, 16, 64, 64, 3)
images_pred_train.shape: (53336, 15, 64, 64, 3)
times_curr_test.shape: (4467,)
images_log_test.shape: (4467, 16, 64, 64, 3)
images_pred_test.shape: (4467, 15, 64, 64, 3)
--------------------------------------------------
image side length: 64
number of log frames: 16
number of pred frames: 15
number of color channels: 3
context(log) image dimension: [16, 64, 64, 3]
future(pred) image dimension: [15, 64, 64, 3]


## Preprocessing the data

In [17]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.float()),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Example normalization
])

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
batch_size_train = 16
batch_size_test = 64
if device.type == "cpu":
  batch_size_train = 4
  batch_size_test = 4
print(batch_size_train, batch_size_test)

cpu
4 4


In [19]:
# Create Dataset and DataLoader
train_dataset = VideoDataset(dss['trainval']['images_log'], dss['trainval']['images_pred'], transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)

test_dataset = VideoDataset(dss['test']['images_log'], dss['test']['images_pred'], transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=False)

# Training

## Defining training & evaluate

## Training

In [20]:
nepochs = 50
print('nepochs:',nepochs)
print_every = 1
eval_every = 5

nepochs: 50
